In [2]:
import glob

import pandas as pd
import hydra
import torch
from torch.utils.data import Dataset

from erc.datasets import KEMDy19Dataset

In [4]:
with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config")
cfg

{'BASE_PATH': '../data/KEMDy19', 'ANN_PATH': '../data/KEMDy19/annotation/'}

In [5]:
dataset = KEMDy19Dataset(cfg=cfg)

AttributeError: 'function' object has no attribute 'glob'

In [5]:
class KEMDy19Dataset(Dataset):
    def __init__(self, cfg):
        MALE_ANN_PATH = '{}/annotation/Session*_M_*'.format(cfg['BASE_PATH'])
        FEMALE_ANN_PATH = '{}/annotation/Session*_F_*'.format(cfg['BASE_PATH'])

        male_annotations = sorted(glob.glob(MALE_ANN_PATH))
        female_annotations = sorted(glob.glob(FEMALE_ANN_PATH))

        total_df = pd.DataFrame()
        selected_columns = ['Numb', 'Wav', 'Unnamed: 2', 'ECG', 'Unnamed: 4', 'E4-EDA', 'Unnamed: 6', 'E4-TEMP', 'Unnamed: 8', 'Segment ID', 'Total Evaluation']

        for m_ann, f_ann in zip(male_annotations, female_annotations):
            m_df = pd.read_csv(m_ann).dropna()[selected_columns]
            f_df = pd.read_csv(f_ann).dropna()[selected_columns]

            # Sess01_impro03, Sess01_impro04의 TEMP와 E4-EDA값이 결측
            m_df = m_df[~(m_df['Segment ID'].str.contains('Sess01_impro03'))&~(m_df['Segment ID'].str.contains('Sess01_impro04'))]
            f_df = f_df[~(f_df['Segment ID'].str.contains('Sess01_impro03'))&~(f_df['Segment ID'].str.contains('Sess01_impro04'))]

            # 각 발화의 성별에 대한 감정만 추출
            m_df = m_df[m_df['Segment ID'].str.contains('M')]
            f_df = f_df[f_df['Segment ID'].str.contains('F')]

            # 다시 합쳐서 정렬
            tmp_df = pd.concat([m_df, f_df], axis=0).sort_values('Numb')
            total_df = pd.concat([total_df, tmp_df], axis=0)
    def make_total_df(self)

    def __len__(self):
        return len(total_df)

    def __getitem__(self, idx):
        X, y = None, None
        ######################################TODO######################################
        X = torch.tensor(self.X[idx],dtype=torch.float)
        y = torch.tensor(self.y[idx],dtype=torch.long)
        ################################################################################
        return X, y

SyntaxError: invalid syntax (2059194551.py, line 27)

In [6]:
MALE_ANN_PATH = "{}/annotation/Session*_M_*".format(cfg["BASE_PATH"])
FEMALE_ANN_PATH = "{}/annotation/Session*_F_*".format(cfg["BASE_PATH"])

male_annotations = sorted(glob.glob(MALE_ANN_PATH))
female_annotations = sorted(glob.glob(FEMALE_ANN_PATH))

total_df = pd.DataFrame()
selected_columns = [
    "Numb",
    "Wav",
    "Unnamed: 2",
    "ECG",
    "Unnamed: 4",
    "E4-EDA",
    "Unnamed: 6",
    "E4-TEMP",
    "Unnamed: 8",
    "Segment ID",
    "Total Evaluation",
]

for m_ann, f_ann in zip(male_annotations, female_annotations):
    m_df = pd.read_csv(m_ann).dropna()
    f_df = pd.read_csv(f_ann).dropna()

    # m_df = pd.read_csv(m_ann).dropna()[selected_columns]
    # f_df = pd.read_csv(f_ann).dropna()[selected_columns]

    # Sess01_impro03, Sess01_impro04의 TEMP와 E4-EDA값이 결측
    # m_df = m_df[~(m_df['Segment ID'].str.contains('Sess01_impro03'))&~(m_df['Segment ID'].str.contains('Sess01_impro04'))]
    # f_df = f_df[~(f_df['Segment ID'].str.contains('Sess01_impro03'))&~(f_df['Segment ID'].str.contains('Sess01_impro04'))]

    # 각 발화의 성별에 대한 감정만 추출
    m_df = m_df[m_df["Segment ID"].str.contains("M")]
    f_df = f_df[f_df["Segment ID"].str.contains("F")]

    # 다시 합쳐서 정렬
    tmp_df = pd.concat([m_df, f_df], axis=0).sort_values("Numb")
    total_df = pd.concat([total_df, tmp_df], axis=0)

In [8]:
seg_id = "Sess20_impro04_M019"
session, script_type, speaker = seg_id.split("_")

wav_txt_path_fmt = "~/workspace/data/KEMDy19/wav/Session{0}/Sess{0}_{1}"
eda_path_fmt = "~/workspace/data/KEMDy19/EDA/Session{0}/Original/Sess{0}{1}.csv"

wav_txt_path = wav_txt_path_fmt.format(session[-2:], script_type)
eda_path = eda_path_fmt.format(session[-2:], speaker[0])


print(wav_txt_path)
print(eda_path)

~/workspace/data/KEMDy19/wav/Session20/Sess20_impro04
~/workspace/data/KEMDy19/EDA/Session20/Original/Sess20M.csv
